In [ ]:
#using transformers_env

In [1]:
pip install peft

  Using cached sympy-1.13.1-py3-none-any.whl.metadata (12 kB)
Using cached sympy-1.13.1-py3-none-any.whl (6.2 MB)
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.3
    Uninstalling sympy-1.13.3:
      Successfully uninstalled sympy-1.13.3
Note: you may need to restart the kernel to use updated packages.


In [15]:
import torch
print(torch.cuda.is_available())  # Should print True
print(torch.version.cuda)  # Should print 11.8

True
12.4


In [13]:
pip install torch==2.0.1+cu118 torchvision==0.15.2+cu118 torchaudio==2.0.2+cu118 --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB ? eta -:--:--
     ---------------------------------------- 0.0/2.6 GB 932.9 kB/s eta 0:46:47
     ---------------------------------------- 0.0/2.6 GB 882.6 kB/s eta 0:49:27
     ---------------------------------------- 0.0/2.6 GB 915.5 kB/s eta 0:47:40
     ---------------------------------------- 0.0/2.6 GB 972.7 kB/s eta 0:44:52
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:03
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:45
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:45
     ---------------------------------------- 0.0/2.6 GB 1.1 MB/s eta 0:40:45
     --------------------

ERROR: THESE PACKAGES DO NOT MATCH THE HASHES FROM THE REQUIREMENTS FILE. If you have updated the package versions, please update the hashes. Otherwise, examine the package contents carefully; someone may have tampered with them.
    torch==2.0.1+cu118 from https://download.pytorch.org/whl/cu118/torch-2.0.1%2Bcu118-cp310-cp310-win_amd64.whl#sha256=f58d75619bc96e4322343c030b893613701caa2d6db8017155da226c14171335:
        Expected sha256 f58d75619bc96e4322343c030b893613701caa2d6db8017155da226c14171335
             Got        d07c3a6a4050c083ea93a4430ba74a61e97cb20b1c117703828ae275c5b56f74



In [20]:
import torch
torch.cuda.empty_cache()

In [25]:
import torch
print(torch.cuda.memory_allocated() / 1024**2, "MB allocated")
print(torch.cuda.memory_reserved() / 1024**2, "MB reserved")

10310.3193359375 MB allocated
10436.0 MB reserved


In [14]:
import pandas as pd
import os
import torch
import shutil
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, TrainerCallback

# Check disk space
total, used, free = shutil.disk_usage('D:/')
if free < 5 * 1024**3:
    raise RuntimeError(f"Insufficient disk space: {free/1024**3:.2f}GB free, need ~5GB")

# Check write access
output_dir = r'D:/Python/dating coach/dating_coach/dating_coach_gpt2'
if not os.access(output_dir, os.W_OK):
    raise RuntimeError("No write access to output directory")

# Set PyTorch memory optimization
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Memory logging callback
class MemoryLoggingCallback(TrainerCallback):
    def on_step_end(self, args, state, control, **kwargs):
        print(f"Step {state.global_step}: {torch.cuda.memory_allocated() / 1024**2:.2f} MB")

# Save checkpoint callback
class SaveCheckpointCallback(TrainerCallback):
    def on_save(self, args, state, control, **kwargs):
        try:
            model.save_pretrained(os.path.join(args.output_dir, f'checkpoint-{state.global_step}'))
            tokenizer.save_pretrained(os.path.join(args.output_dir, f'checkpoint-{state.global_step}'))
            print(f"Saved checkpoint at step {state.global_step}")
        except Exception as e:
            print(f"Checkpoint save failed at step {state.global_step}: {e}")

# Clean text
def clean_text(text):
    text = str(text).strip()
    if not text[-1] in [' ', '.', ',', '!', '?']:
        words = text.split()
        text = ' '.join(words[:-1]) if words else text
    return text.replace("your doing", "you're doing").replace("mistake guys make", "mistakes guys make")

# Read CSV data
data = []
csv_path = r'D:/Python/dating coach/formatted_data.csv'  # Adjust
if os.path.isfile(csv_path):
    df = pd.read_csv(csv_path)
    data = [{'title': row['title'], 'text': clean_text(row['text'])} for _, row in df.iterrows()]
else:
    for file in os.listdir(csv_path):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(csv_path, file))
            data.extend([{'title': row['title'], 'text': clean_text(row['text'])} for _, row in df.iterrows()])

# Format samples
formatted_samples = []
for item in data:
    sample = (
        f"[Scenario]: {item['title']}\n"
        f"{item['text']}\n"
        f"[Assistant]: For {item['title'].lower()}, keep it short and casual, like: 'Hey, great meeting you! Free this weekend?' Avoid long, eager messages as they can seem pushy."
    )
    formatted_samples.append(sample)

# Subsample
if len(formatted_samples) > 500:
    formatted_samples = formatted_samples[:500]
dataset = Dataset.from_dict({'text': formatted_samples})

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
special_tokens = {'additional_special_tokens': ['[Scenario]', '[Assistant]']}
tokenizer.add_special_tokens(special_tokens)

# Load model
model = AutoModelForCausalLM.from_pretrained('gpt2')
model.resize_token_embeddings(len(tokenizer))  # Fix lm_head.weight

# Enable gradient checkpointing
model.gradient_checkpointing_enable()

# Clear GPU memory
torch.cuda.empty_cache()

# Tokenize dataset with labels
def tokenize(examples):
    tokenized = tokenizer(
        examples['text'],
        truncation=True,
        padding='max_length',
        max_length=128  # Reduced for speed
    )
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized
tokenized_dataset = dataset.map(tokenize, batched=True)

# Split dataset
train_dataset = tokenized_dataset.shuffle().select(range(int(0.8 * len(tokenized_dataset))))
eval_dataset = tokenized_dataset.shuffle().select(range(int(0.8 * len(tokenized_dataset)), len(tokenized_dataset)))

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=10,  # Increased for better learning
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-5,  # Standard for GPT-2
    warmup_steps=50,  # Increased for ~1,000 samples
    max_grad_norm=1.0,  # Gradient clipping
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=50,
    save_strategy='steps',
    save_steps=50,
    save_total_limit=3,
    save_safetensors=False,  # Use pytorch_model.bin
    load_best_model_at_end=True,
    fp16=True,
    gradient_checkpointing=True,
    lr_scheduler_type='linear',  # Smooth learning rate decay
    report_to='none'
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)
trainer.add_callback(MemoryLoggingCallback())
trainer.add_callback(SaveCheckpointCallback())

# Train
try:
    trainer.train()
except Exception as e:
    print(f"Training failed: {e}")
    try:
        model.save_pretrained(os.path.join(output_dir, 'final'))
        tokenizer.save_pretrained(os.path.join(output_dir, 'final'))
    except Exception as save_e:
        print(f"Save failed: {save_e}")
    raise

# Save model and tokenizer
try:
    model.save_pretrained(os.path.join(output_dir, 'final'))
    tokenizer.save_pretrained(os.path.join(output_dir, 'final'))
except Exception as e:
    print(f"Final save failed: {e}")


Map: 100%|██████████| 500/500 [00:00<00:00, 1508.92 examples/s]


Step,Training Loss,Validation Loss
50,3.407100,3.149776
100,2.948600,2.719949
150,2.744700,2.570954
200,2.606300,2.483197
250,2.489300,2.425578
300,2.472000,2.388641
350,2.285800,2.357845
400,2.433300,2.340310
450,2.373300,2.329093
500,2.388500,2.325120


Step 1: 508.62 MB
Step 2: 1463.92 MB
Step 3: 1463.92 MB
Step 4: 1463.92 MB
Step 5: 1463.92 MB
Step 6: 1463.92 MB
Step 7: 1463.92 MB
Step 8: 1463.92 MB
Step 9: 1463.92 MB
Step 10: 1463.92 MB
Step 11: 1463.92 MB
Step 12: 1463.92 MB
Step 13: 1463.92 MB
Step 14: 1463.92 MB
Step 15: 1463.92 MB
Step 16: 1463.92 MB
Step 17: 1463.92 MB
Step 18: 1463.92 MB
Step 19: 1463.92 MB
Step 20: 1463.92 MB
Step 21: 1463.92 MB
Step 22: 1463.92 MB
Step 23: 1463.92 MB
Step 24: 1463.92 MB
Step 25: 1463.92 MB
Step 26: 1463.92 MB
Step 27: 1463.92 MB
Step 28: 1463.92 MB
Step 29: 1463.92 MB
Step 30: 1463.92 MB
Step 31: 1463.92 MB
Step 32: 1463.92 MB
Step 33: 1463.92 MB
Step 34: 1463.92 MB
Step 35: 1463.92 MB
Step 36: 1463.92 MB
Step 37: 1463.92 MB
Step 38: 1463.92 MB
Step 39: 1463.92 MB
Step 40: 1463.92 MB
Step 41: 1463.92 MB
Step 42: 1463.92 MB
Step 43: 1463.92 MB
Step 44: 1463.92 MB
Step 45: 1463.92 MB
Step 46: 1463.92 MB
Step 47: 1463.92 MB
Step 48: 1463.92 MB
Step 49: 1463.92 MB
Step 50: 1463.92 MB
Saved chec